In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
from datasets import load_dataset
import pandas as pd
import itertools
import os
from datetime import datetime


data_path = '/home/jovyan/zhang/Husky/data/2022/sample/infotree_mrsmr_000_20240116_114018_149346.json'
data = load_dataset('json', data_files=data_path)

df = pd.DataFrame(data["train"])

In [15]:
result = []
for index, items in df.iterrows():
    
    for item in items['smrs']:
        if item['smr'] != None:
            item['INPUT_DATE'] = datetime.strptime(item['INPUT_DATE'], '%Y-%m-%d')
    for item in items['records']:
        item['ts'] = datetime.strptime(item['ts'][:8], '%Y%m%d')
        
    smrs = items['smrs']
    records = items['records']
    
    for smrs_item in smrs:
        # 找到第一个 records 元素的时间戳晚于当前 smrs 元素的时间戳的位置
        insert_index = next((i for i, record_item in enumerate(records) if record_item['ts'] > smrs_item['INPUT_DATE']), len(smrs))
    
        # 将列表截断，并插入 smrs 元素
        result.append({'pt_id': items['pt_id'], 'records': records[:insert_index], 'smrs':smrs_item})
        records = records[insert_index:]          # 更新剩余的 records 元素列表
        if len(records)==0:
            break

In [16]:
df_result = pd.DataFrame(result)

In [19]:
df_result = df_result[df_result['records'].apply(lambda x: len(x)>0)]

In [35]:
df_result

In [60]:
records = []
smrs = []
for index, items in df_result.iterrows():
    for record in items['records']:
        for line in record['emr']:
            line['pt_id'] = items['pt_id']
            line['hcp_class'] = record['hcp_class']
            line['section'] = record['section']
            records.append(line)

    for smr in items['smrs']['smr']:
        smr['pt_id'] = items['pt_id']
        smr['EMR_TYPE'] = items['smrs']['EMR_TYPE']
        smr['SECTION'] = items['smrs']['SECTION']
        smrs.append(smr)

In [61]:
df_records = pd.DataFrame(records)
df_smrs = pd.DataFrame(smrs)

In [76]:
grouped_df_records = df_records.groupby('pt_id')
grouped_df_smrs = df_smrs.groupby('pt_id')

In [66]:
vectorizer = TfidfVectorizer()
tfidf_matrix_records = vectorizer.fit_transform(df_records['text'].tolist())
tfidf_matrix_smrs = vectorizer.transform(df_smrs['EMR_TEXT'].tolist())

In [67]:
similarity_matrix = cosine_similarity(tfidf_matrix_records, tfidf_matrix_smrs)

In [74]:
df_records

In [75]:
df_smrs

In [69]:
import numpy as np
closest_sentences = []
for i in range(similarity_matrix.shape[0]):
    closest_index = np.argmax(similarity_matrix[i])  # 获取集合B中最相似句子的索引
    closest_sentences.append((df_records['text'][i], df_smrs['EMR_TEXT'][closest_index]))

In [73]:
closest_sentences[10]